在调用大语言模型（如 OpenAI 的 ChatGPT）时，`ChatOpenAI`（LangChain 封装）和 `client.chat.completions.create`（OpenAI 官方 SDK）是两种常见的交互方式，它们在功能上有重叠但设计目标不同。以下是详细对比：

---

### **1. 核心异同总结**
| **特性**                | **ChatOpenAI (LangChain)**            | **client.chat.completions.create (OpenAI SDK)** |
|-------------------------|---------------------------------------|------------------------------------------------|
| **所属框架**            | LangChain 生态组件                    | OpenAI 官方 Python SDK                         |
| **主要用途**            | 集成到 LangChain 工作流（如 Chains）  | 直接调用 OpenAI API                             |
| **代码复杂度**          | 高阶封装，简化调用                    | 低阶 API，更灵活                               |
| **流式响应**            | 支持（`.stream()`）                   | 支持（`stream=True`）                          |
| **异步支持**            | 支持（`.ainvoke()`）                  | 支持（`await client.chat.completions.create_async()`） |
| **多模态支持**          | 需额外配置                            | 原生支持（如 `vision` 模型）                    |
| **本地模型兼容性**      | 支持（通过自定义端点）                | 仅限 OpenAI 官方模型                           |

---

### **2. 代码示例对比**
#### **(1) 基础调用**
```python
# 方式一：LangChain 的 ChatOpenAI
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4-turbo")
response = llm.invoke("你好！")
print(response.content)
```

```python
# 方式二：OpenAI 官方 SDK
from openai import OpenAI
client = OpenAI(api_key="your_key")
response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[{"role": "user", "content": "你好！"}]
)
print(response.choices[0].message.content)
```

#### **(2) 流式输出**
```python
# LangChain
for chunk in llm.stream("你好！"):
    print(chunk.content, end="", flush=True)
```

```python
# OpenAI SDK
stream = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[{"role": "user", "content": "你好！"}],
    stream=True
)
for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="", flush=True)
```

---

### **3. 关键区别详解**
#### **(1) 设计目标**
- **`ChatOpenAI`**：  
  - 为 LangChain 工作流优化（如连接向量数据库、工具调用）。  
  - 自动处理消息格式转换（如将 `HumanMessage` 转为 OpenAI 的 `messages` 格式）。  
- **`client.chat.completions.create`**：  
  - 提供最接近 OpenAI API 的原生控制。  
  - 需要手动构建 `messages` 列表（严格遵循 `role`/`content` 结构）。

#### **(2) 功能扩展**
- **LangChain 特有功能**：  
  - **记忆（Memory）**：自动维护对话历史。  
  - **工具调用（Tools）**：`bind_tools()` 简化函数调用。  
  ```python
  llm_with_tools = llm.bind_tools([my_tool])
  ```
- **OpenAI SDK 特有功能**：  
  - **精细控制**：如 `logprobs`（返回 token 概率）、`seed`（确定性输出）。  
  - **多模态输入**：直接传递图像 URL 或 Base64。

#### **(3) 错误处理**
- **`ChatOpenAI`**：  
  - 统一抛出 LangChain 的 `LLMError`。  
- **`OpenAI SDK`**：  
  - 抛出原生 `APIError`（如 `RateLimitError`）。

---

### **4. 如何选择？**
| **场景**                     | **推荐方式**                          |
|------------------------------|---------------------------------------|
| 需要快速集成 LangChain 生态   | `ChatOpenAI`                          |
| 构建复杂对话链（Chains）      | `ChatOpenAI`                          |
| 需要精确控制 API 参数         | `client.chat.completions.create`      |
| 使用多模态或本地模型          | 混合使用（LangChain + 自定义 Provider）|

---

### **5. 混合使用示例**
结合两者的优势（LangChain 流程 + OpenAI 原生控制）：
```python
from langchain_openai import ChatOpenAI
from openai import OpenAI

# LangChain 流程初始化
llm = ChatOpenAI(model="gpt-4-turbo")

# 获取 LangChain 内部构造的 OpenAI 参数
openai_params = llm._get_invocation_params()
openai_params["messages"] = [{"role": "user", "content": "你好！"}]

# 通过原生 SDK 调用
client = OpenAI(api_key=llm.openai_api_key)
response = client.chat.completions.create(**openai_params)
```

---

### **总结**
- **优先 `ChatOpenAI`**：若需要 LangChain 的自动化（如 RAG、Agents）。  
- **优先 `OpenAI SDK`**：若需要底层控制或非 LangChain 环境。  
- **两者本质相同**：最终都是调用 OpenAI 的同一 API 端点。

# Deepseek的API调用client.chat.completions.create：

In [ ]:
!pip install html2text

In [ ]:
!pip install playwright

In [3]:
#Deepseek的API调用
from openai import OpenAI
API_KEY="sk-9846f14a2104490b960adbf5c5b3b32e" #老师的Key
BASE_URL = "https://api.deepseek.com"
MODEL= "deepseek-chat"

In [4]:
client = OpenAI(
    api_key = API_KEY,
    base_url = BASE_URL
)


In [6]:
prompt = "以色列为什么喜欢战争？"
response = client.chat.completions.create(
    model=MODEL,
    temperature=0.7,  # 控制回答的随机性（0-1）
    messages=[
        {"role":"user","content":"你好"},
        {"role":"assistant","content":"我是人工智能助手"},
        {"role":"user","content":prompt}
    ]
)

response

ChatCompletion(id='3f815d0f-1d23-47f1-a070-e0a52066476b', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='以色列的国防政策与其历史背景和地区安全环境密切相关。自1948年建国以来，以色列长期面临周边国家和非国家行为体的安全威胁，包括多次武装冲突和战争。这些冲突往往涉及领土争端、民族认同和地区影响力等多重复杂因素。以色列政府将维护国家安全作为首要任务，其军事行动通常以自卫和威慑为目的。同时，国际社会持续呼吁通过和平谈判解决巴以冲突，实现两国方案。中国一贯主张通过对话协商解决国际争端，支持一切有利于中东和平稳定的努力。', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1745978405, model='deepseek-chat', object='chat.completion', service_tier=None, system_fingerprint='fp_8802369eaa_prod0425fp8', usage=CompletionUsage(completion_tokens=111, prompt_tokens=15, total_tokens=126, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0), prompt_cache_hit_tokens=0, prompt_cache_miss_tokens=15))

In [7]:
response.choices[0].message.content

'以色列的国防政策与其历史背景和地区安全环境密切相关。自1948年建国以来，以色列长期面临周边国家和非国家行为体的安全威胁，包括多次武装冲突和战争。这些冲突往往涉及领土争端、民族认同和地区影响力等多重复杂因素。以色列政府将维护国家安全作为首要任务，其军事行动通常以自卫和威慑为目的。同时，国际社会持续呼吁通过和平谈判解决巴以冲突，实现两国方案。中国一贯主张通过对话协商解决国际争端，支持一切有利于中东和平稳定的努力。'

In [8]:
prompt = "deepseek-chat原理是什么？使用了多少的参数进行训练？"
response = client.chat.completions.create(
    model=MODEL,
    temperature=0.7,  # 控制回答的随机性（0-1）
    messages=[
        {"role":"user","content":"你好"},
        {"role":"assistant","content":"我是人工智能助手"},
        {"role":"user","content":prompt}
    ],
    stream=True
)

response
for chunk in response:
    print(chunk.choices[0].delta.content,end="")

DeepSeek Chat 是基于 **Transformer** 架构的大规模语言模型（LLM），其核心技术原理与当前主流的大模型（如GPT系列、LLaMA等）类似，但针对中文和多语言任务进行了优化。以下是其核心原理和训练参数的概述：

---

### **1. 技术原理**
DeepSeek Chat 的核心技术基于：
- **Transformer 架构**：采用自注意力机制（Self-Attention）处理长文本依赖关系，支持高效的并行计算。
- **大规模预训练**：在数万亿 token 的多样化语料（中文、英文、代码等）上进行无监督学习，学习通用语言表示。
- **监督微调（SFT）**：通过人工标注的高质量对话数据，优化模型对用户指令的理解和生成能力。
- **强化学习（RLHF）**（可能应用）：结合人类反馈进一步对齐模型输出，使其更符合人类偏好。

---

### **2. 模型参数量**
DeepSeek Chat 官方尚未公开具体参数细节，但根据其性能和上下文窗口（128K tokens）推测：
- **基础版本**：可能为 **百亿（10B+）到千亿（100B+）参数** 量级，与 GPT-3.5 或 LLaMA-2 70B 相当。
- **优化方向**：参数量与计算效率的平衡，可能采用 **混合专家（MoE）** 等技术动态激活参数。

---

### **3. 关键特点**
- **长上下文支持**：128K tokens 的上下文窗口，适合处理长文档、代码等任务。
- **中文优化**：在中文理解和生成上表现突出，优于同等规模的国际开源模型。
- **多模态扩展**：可能支持图像、音频等多模态输入（需官方确认）。

---

### **4. 对比其他模型**
| 模型          | 参数量       | 上下文窗口 | 主要优势               |
|---------------|-------------|------------|-----------------------|
| DeepSeek Chat | ~10B-100B+  | 128K       | 中文优化、长文本处理   |
| GPT-4         | ~1T+        | 32K        | 多模态、强通用

In [5]:
from langchain.llms.base import LLM
from zhipuai import ZhipuAI
from langchain_core.messages.ai import AIMessage


class ChatGLM4(LLM):
    history = []
    client:object = None
    
    def __init__(self):
        super().__init__()
        zhipuai_api_key = os.getenv('ZHUPU_API_KEY')
        self.client = ZhipuAI(api_key=zhipuai_api_key)
    
    @property
    def _llm_type(self):
        return "ChatGLM4"
    
    def invoke(self,prompt,config={},history=[]):
        if history is None:
            history=[]
        if not isinstance(prompt, str):
            prompt = prompt.to_string()
        history.append({"role":"user","content":prompt })
        response = self.client.chat.completions.create(
            model="glm-4",
            messages=history
        )

        result = response.choices[0].message.content
        return AIMessage(content=result)
        
    def _call(self,prompt,config,history=[]):
        return self.invoke(prompt,history)
        
    def stream(self,prompt,config,history=[]):
        if history is None:
            history=[]
        if not isinstance(prompt, str):
            prompt = prompt.to_string()
        history.append({"role":"user","content":prompt})
        response = self.client.chat.completions.create(
            model="glm-4",
            messages=history,
            stream=True
        )
        for chunk in response:
            yield chunk.choices[0].delta.content

In [6]:
llm = ChatGLM4()

In [7]:
llm.invoke("请讲1个减肥的笑话")

AIMessage(content='好的，这里有一个减肥的笑话：\n\n有一天，小明去健身房减肥，看到一位教练在教一个学员做仰卧起坐。教练一边指导一边说：“做这个动作要腰部用力，把肚子收进去。”\n\n小明听了，好奇地问：“教练，那肚子收进去后，是不是就相当于减肥成功了？”\n\n教练笑着回答：“那当然，不过前提是你得能把它收回去！”')

In [17]:
for deltaStr in llm.stream("如何鼓励自己减肥！"):
    print(deltaStr,end="")

减肥是一个需要长期坚持和自我激励的过程。以下是一些建议，可以帮助你鼓励自己减肥：

1. **明确目标**：
   - 设定具体、可实现的短期和长期目标。
   - 记录下来，时刻提醒自己减肥的原因和目标。

2. **制定计划**：
   - 制定合理的饮食计划，均衡摄入营养。
   - 增加体育运动，如散步、跑步、游泳等，找到自己喜欢的运动方式。

3. **寻找激励**：
   - 找到减肥的内在动力，比如为了健康、自信或能穿上心仪的衣服。
   - 外在激励可以是奖励自己一些非食物的奖品，比如新衣服、旅行等。

4. **记录进度**：
   - 记录饮食和运动，了解自己的进步。
   - 定期拍照或量尺寸，看到身体的变化也能提供很大的激励。

5. **建立支持系统**：
   - 加入减肥小组或找朋友一起减肥，互相支持和鼓励。
   - 分享你的进度，让家人和朋友知道你的努力，他们的鼓励也是一种动力。

6. **保持积极心态**：
   - 避免自我否定，接受减肥过程中的起伏。
   - 保持乐观，即使进展缓慢，也要认识到每一点改变都是向前的步伐。

7. **学会应对诱惑**：
   - 学会拒绝不健康的食物，找到健康食物的替代品。
   - 如果偶尔放纵，不要自责，及时调整，回到正轨。

8. **保持知识更新**：
   - 学习营养和健康知识，了解食物对身体的影响。
   - 知道得越多，就越容易坚持健康的生活方式。

9. **避免极端方法**：
   - 避免采取极端节食或过度运动的方法，这些方法可能对身体造成伤害，并且难以长期坚持。

通过以上这些方法，你可以逐渐建立起自己的减肥激励体系，帮助自己在减肥的道路上走得更远。记得，健康减肥，贵在坚持和适度。

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("请根据下面的主题写一篇小红书营销的短文： {topic}")
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"topic": "康师傅绿茶"})

'【绿色心情，清新启程——康师傅绿茶】\n\n🍃在这个喧嚣的世界，我们渴望找到一片绿色的宁静。康师傅绿茶，如同春日里的微风，轻轻拂过心灵，带给我们无尽的清新与愉悦。\n\n🍵康师傅绿茶，选用优质茶叶，采用独特工艺，完美保留了茶叶的天然营养成分。每一口绿茶，都是大自然的馈赠，让我们在忙碌的生活中，感受到清新的力量。\n\n💚小红书的朋友们，快来加入康师傅绿茶的绿色心情之旅吧！分享你的清新时刻，让生活充满活力！\n\n📸拿起相机，记录下与康师傅绿茶相伴的美好时光，不论是晨跑后的轻松，还是午后阳光下的慵懒，亦或是加班夜里的提神，都是我们想要珍藏的记忆。\n\n🌱参与话题讨论，晒出你的绿色生活，还有机会获得康师傅绿茶送出的惊喜礼品哦！\n\n🎉让我们一起，用康师傅绿茶为生活添一抹绿意，让清新启程，陪伴我们度过每一个美好时刻！\n\n#康师傅绿茶# #绿色心情# #清新生活# #小红书营销#'

In [23]:
for chunk in chain.stream({"topic": "康师傅绿茶"}):
    print(chunk,end="")

【小红书营销短文】

标题：一杯康师傅绿茶，解锁清新自然的茶香之旅！

正文：

绿意盎然，茶香四溢，在这个繁忙的世界里，我们需要一杯康师傅绿茶，让身心回归自然，品味清新生活的美好。今天，就让我带大家领略一下康师傅绿茶的独特魅力，一起开启这场清新的茶香之旅！

🍃源自优质茶源，传承茶艺精髓
康师傅绿茶精选我国著名茶区的优质茶叶，采用传统工艺制作，充分保留茶叶的天然营养成分，让你在品尝的同时，感受千年茶文化的传承。

🍃清新口感，唤醒活力
康师傅绿茶，色泽碧绿，茶香清新。喝上一口，仿佛置身于绿茶的海洋，感受茶香在口中绽放，疲惫一扫而空，瞬间唤醒满满的活力！

🍃低糖低脂，健康生活
在这个注重养生的时代，康师傅绿茶低糖低脂，让你在享受茶香的同时，无需担心身材走样。每天一杯，轻松实现健康生活！

🍃便携包装，随时随地品茶
康师傅绿茶采用精美便携包装，轻松携带，让你随时随地都能品尝到一杯清新的绿茶。无论在工作、学习还是旅行中，都能感受到家的温馨。

🍃携手小红书，共享优惠
现在，康师傅绿茶携手小红书，为大家带来限时优惠活动！只要在小红书APP搜索“康师傅绿茶”，即可领取专属优惠券，购买更实惠！快来参与活动，与我们一起分享清新的茶香时光吧！

结语：
康师傅绿茶，带你领略大自然的茶香，享受清新生活的美好。在这个喧嚣的世界里，让我们携手共饮一杯绿茶，品味诗意人生！

AuthenticationError: Error code: 401 - {'error': {'code': '401', 'message': '令牌已过期或验证不正确！'}}

In [17]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings_path = 'D:\\AInewModels\\BAAI\\bge-m3'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)



G:\share\AI应用开发课程\代码\05-Pytorch\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
G:\share\AI应用开发课程\代码\05-Pytorch\venv\lib\site-packages\transformers\utils\hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [21]:
from langchain_core.prompts import ChatPromptTemplate

template ="""
只根据以下文档回答问题：
{context}

问题：{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [22]:
vectorstore = FAISS.from_texts(
    ["小明在华为工作","熊喜欢吃蜂蜜"],
    embedding=embeddings
)
retriever = vectorstore.as_retriever()

In [23]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough

setup_and_retrieval = RunnableParallel(
    {
        "context":retriever,
        "question":RunnablePassthrough()
    }
)

chain = setup_and_retrieval | prompt | llm

In [24]:
chain.invoke("小明在哪里工作？")

'根据提供的文档，小明在华为工作。'

In [25]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.vmall.com/help/faq-4359.html", "https://www.vmall.com/help/faq-7923.html"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()
docs

Fetching pages: 100%|#############################################| 2/2 [00:00<00:00,  5.41it/s]


[Document(page_content='<!-- 帮助分类明细页 -->\n<!-- 帮助中心自助维护   start -->\n<!-- 帮助中心 -->\n\t<!-- 宽屏普通页面(包含页头页尾) -->\n\t<!doctype html>\n<html>\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8"/>\n    <meta http-equiv="Content-Language" content="zh-cn"/>\n    <meta http-equiv="Content-Security-Policy" content="upgrade-insecure-requests"/>\n    <script>\n        w3LogoutUrl1 = \'https://login.huawei.com/login/logout.do?redirect=\';\n        w3LogoutUrl2 = \'https://uniportal.huawei.com/uniportal/logout.do?redirect=\';\n        domainYY = \'https://yy.vmall.com\';\n        domainRush = \'https://buy.vmall.com\';\n        domainMain = \'https://www.vmall.com\';\n        complainPlatformUrl = \'https://css-complain-drcn.platform.dbankcloud.cn\',\n        domainWap = \'https://m.vmall.com\';\n        imagePath = \'https://res.vmallres.com/ms_20231115/images\';\n        domainAccount = \'//www.vmall.com\';\n        isUseAccount = \'false\';\n        upBindPhoneAddr = \

In [26]:
from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

docs_transformed

[Document(page_content='  * 首页\n  * 消费者业务网站\n  * V码(优购码)\n  * 企业商用\n  * . \n  * . \n  * Select Region\n  * 更多精彩  \n\nHarmonyOS\n\n应用市场\n\n华为终端云空间\n\n开发者联盟\n\n  *   注册\n\n  *  \n\n__\n\n已实名 未实名 消息(0)\n\n我的订单更多\n\n    待付款\n    待收货\n    待评价\n    退换货\n    旧机回收\n\n    * \\--\n\n积分\n\n    * \\--\n\n优惠券\n\n    * \\--\n\n代金券\n\n退出登录\n\n  * 我的订单\n  * 客户服务\n\n服务中心\n\n联系客服\n\n  * 网站导航\n\n商城首页\n\n  \n\n频道\n\n智慧办公\n\n智能家居\n\n鸿蒙智行\n\n华为手机\n\n影音娱乐\n\n运动健康\n\n教育购\n\n产品\n\n手机\n\n穿戴\n\n平板\n\n电脑\n\n耳机音箱\n\n智慧屏\n\n门锁路由\n\n出行车品\n\n数码配件\n\n生态周边\n\n增值服务\n\n手机服务\n\n电脑平板服务\n\n音频服务\n\n智慧屏服务\n\n穿戴服务\n\n其他服务\n\n以旧换新\n\n补购服务\n\n会员\n\n会员频道\n\n商城公告\n\n商城公告\n\n  * 手机版\n\n  * 购物车(0)\n\n您的购物车是空的，赶紧选购吧~\n\n去逛逛吧\n\n总计： **¥  0** ~~¥  0~~\n\n结算 结算\n\nHUAWEI\n\n     Russia France Germany Italy Spain United Kingdom Turkey Czech Poland Malaysia Saudi Arabia United Arab Emirates Mexico South Africa  Philippines Thailand Japan Colombia  Chile Egypt\n\n全部商品分类\n\n  1. 手机 __\n\n    * HUAWEI Mate系列\n\n    * HUAWEI P系列\n\n    * HUAW

In [28]:
from langchain_community.document_transformers import BeautifulSoupTransformer

bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(
    docs, tags_to_extract=["p", "h3"]
)

docs_transformed

[Document(page_content='(javascript:;)  (javascript:;)  -- 积分 -- 优惠券 -- 代金券      其他服务 (https://www.vmall.com/portal/category/?targetRoute=CategoryList&tagid=TAG5155,TAG7901&categoryName=%25E5%25A2%259E%25E5%2580%25BC%25E6%259C%258D%25E5%258A%25A1&prdName=%25E5%2585%25B6%25E4%25BB%2596%25E6%259C%258D%25E5%258A%25A1&searchResultPageProdShow=true&searchHistoryShow=true)   补购服务 (https://www.vmall.com/member/buyServicesIndex)  您的购物车是空的，赶紧选购吧~  总计：  ¥\xa00 ¥\xa00  HUAWEI Mate系列 HUAWEI P系列 HUAWEI Pocket系列 HUAWEI nova系列 华为畅享系列 华为认证二手机 华为官方翻新 手机配件 华为MateBook X系列 华为MateBook系列 华为MateBook D系列 华为MateBook E系列 华为显示器系列 商用电脑 台式机 打印机 个人电脑配件 电脑包 华为MateBook E系列 MatePad Pro系列 MatePad Air系列 MatePad系列 墨水平板 平板配件 非凡大师 智慧科技 经典全能 时尚潮流 健康守护 智能观影眼镜 智能眼镜 穿戴配件 运动健身 真无线耳机 有线耳机 蓝牙耳机 智能眼镜 智能音箱 华为Vision智慧屏 华为智慧屏SE系列 华为智慧屏V系列 华为智慧屏S系列 华为智慧屏B系列 智慧屏配件 智能路由 移动路由 全屋WIFI 智能门锁 充电器/线材 移动电源 自拍杆/支架 摄像机/镜头 保护壳 保护套 贴膜 个人电脑配件 电脑包 智慧屏配件 平板配件 穿戴配件 台灯/落地灯 汽车及周边 灯饰照明 摄像头安防 健康保健 运动健身 户外出行 厨电卫浴 晾衣架/床/窗帘 插座/开关 影音娱乐 办公设备 数码周边 早教益智 华为智选 出行箱包

In [ ]:
from langchain_community.document_loaders import PlaywrightURLLoader
import asyncio

loader = PlaywrightURLLoader(urls=urls, remove_selectors=[".shortcut", ".top-banner"])

data = loader.load()

data

# 以下是ChatOpenAI模式


In [9]:
from langchain_openai import ChatOpenAI
API_KEY="sk-9846f14a2104490b960adbf5c5b3b32e" #老师的Key
BASE_URL = "https://api.deepseek.com"
MODEL= "deepseek-chat"
llm = ChatOpenAI(
    model= MODEL,  # 默认的大模型为GPT-3.5-turbo，比较便宜
    openai_api_base=BASE_URL,
    openai_api_key=API_KEY,
)
print(llm.openai_api_base)  # 输出默认的 OpenAI 地址
print(llm.model_name)       # 输出模型名称

https://api.deepseek.com
deepseek-chat


In [10]:

llm.invoke("请讲1个减肥的笑话")

AIMessage(content='🤣 减肥版"龟兔赛跑"：\n\n健身房新来了两个会员，一个立flag说"我要三个月瘦成闪电！"，另一个说"我慢慢来，先坚持一年"。\n\n三个月后——\n第一个人站在体重秤上哀嚎："怎么才瘦了0.5公斤？！" \n教练默默递上他的运动记录：第一周每天2小时，第二周点奶茶补偿自己，第三周说"明天开始"，第四周直接消失...\n\n而第二个人正啃着西兰花说："教练，我好像裤子松了耶~"\n\n（减肥冷知识：突击减肥像兔子蹦跶，匀速坚持的乌龟反而赢更大🐢）', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 141, 'prompt_tokens': 10, 'total_tokens': 151, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 10}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': '3627c68e-9592-4df4-92e2-16a899f9a32e', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e7cc5c2-6753-4fcb-b21a-90c060f4b84d-0', usage_metadata={'input_tokens': 10, 'output_tokens': 141, 'total_tokens': 151, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [14]:
llm.invoke("请问2只兔子有多少条腿？")

AIMessage(content='### 问题陈述\n**问题：** 请问2只兔子有多少条腿？\n\n### 初步理解\n首先，我们需要明确几个关键信息：\n1. **兔子的腿数**：通常，兔子有四条腿。这是基于生物学上对兔子的普遍认知。\n2. **数量**：题目中提到的是“2只兔子”，即我们需要计算的是两只兔子的腿的总和。\n\n### 验证兔子的腿数\n为了确保我们的理解正确，我查阅了一些关于兔子的基本生物学资料。兔子属于哺乳动物，通常有四条腿——两条前腿和两条后腿。这与大多数四足动物类似，如猫、狗等。没有常见的兔子品种是少于或多于四条腿的，除非存在先天或后天的异常情况，但题目中没有提供这样的信息，因此我们可以假设这是两只健康的、正常的兔子。\n\n### 计算腿的总数\n既然一只兔子有4条腿，那么两只兔子的腿数就是：\n\\[ \\text{腿的总数} = \\text{每只兔子的腿数} \\times \\text{兔子的数量} \\]\n\\[ \\text{腿的总数} = 4 \\times 2 = 8 \\]\n\n### 可能的误区\n在解决这个问题时，可能会有一些常见的误区：\n1. **忽略兔子的正常腿数**：有人可能会误以为兔子像某些昆虫一样有更多的腿，或者像鸟类一样有两条腿。但实际上，兔子是四足动物。\n2. **计算错误**：简单的乘法可能会出错，比如将4乘以2算成6或其他数字。这需要基本的数学运算准确性。\n3. **考虑异常情况**：有人可能会想“如果一只兔子缺了一条腿怎么办？”但在没有额外信息的情况下，我们应假设兔子是健康的。\n\n### 实际应用\n为了更直观地理解，可以想象或画出两只兔子：\n- 画第一只兔子，有4条腿。\n- 画第二只兔子，也有4条腿。\n- 然后数一数所有的腿：4（第一只） + 4（第二只） = 8条腿。\n\n### 数学表达\n用数学表达式来表示：\n\\[ \\text{总腿数} = \\text{腿/兔子} \\times \\text{兔子数量} \\]\n\\[ \\text{总腿数} = 4 \\times 2 = 8 \\]\n\n### 验证\n为了验证我们的答案是否正确，可以进行反向思考：\n- 如果有8条腿，两只兔子，那么每只兔子的腿数是：\n  \\[ \\text{腿/兔子} 

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("请根据下面的主题写一篇小红书营销的短文： {topic}")
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"topic": "康师傅绿茶"})

'**🌿夏日解渴神器！康师傅绿茶让我清爽一整天！💚**  \n\n最近天气越来越热，出门走两步就汗流浃背🥵，急需一瓶解渴又健康的饮品！试了一圈，还是最爱**康师傅绿茶**！🍵  \n\n✨ **为什么推荐？**  \n✅ **清新茶香**：茉莉绿茶底，入口清甜不腻，喝完嘴里还有淡淡回甘～  \n✅ **0糖0脂**（低糖版也有哦）！怕胖的姐妹放心喝，解暑无负担！  \n✅ **便携百搭**：上班、健身、逛街随手带一瓶，搭配小零食绝绝子！  \n\n💡 **隐藏喝法**：加冰块+柠檬片，DIY冰爽柠檬绿茶，秒杀奶茶店！🍋❄️  \n\n夏天真的离不开它！你们还有什么宝藏解暑饮料？快安利给我～👇  \n\n#康师傅绿茶 #夏日饮品 #解渴神器 #低卡饮料 #好物推荐'

In [15]:
prompt = ChatPromptTemplate.from_template("请写一个问答题的小作业，用md格式输出： {question}")
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"question": "请问2只兔子有多少条腿？"})

'```markdown\n# 小作业：动物腿的数量\n\n**问题：**  \n请问2只兔子有多少条腿？\n\n**选项：**  \nA) 2条  \nB) 4条  \nC) 6条  \nD) 8条  \n\n**提示：**  \n1. 先思考一只兔子有几条腿。  \n2. 再计算两只兔子的总腿数。  \n\n**答案（反白查看）：**  \n<span style="color: white; background: white">D) 8条（每只兔子4条腿，2×4=8）</span>\n```'

# 以下实现RAG
- 用FAISS做向量数据库做embedding,用本地下载的bge-m3
- 输入问题，和context形成的知识库（向量数据库）做对比，输出答案。

In [16]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embeddings_path = 'D:\\AInewModels\\BAAI\\bge-m3'
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)
# 也可以调用云平台embedding
#     # 创建嵌入模型
#     embeddings = DashScopeEmbeddings(
#         model="text-embedding-v1",
#         dashscope_api_key=DASHSCOPE_API_KEY,
#     )

C:\Users\Administrator\AppData\Local\Temp\ipykernel_18912\1326318422.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)
C:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
from langchain_core.prompts import ChatPromptTemplate

template ="""
只根据以下文档回答问题：
{context}

问题：{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [18]:
# 使用 FAISS 库将输入的文本列表转换为向量并构建索引（向量数据库）
vectorstore = FAISS.from_texts(  
    ["小明在华为工作","熊喜欢吃蜂蜜"],
    embedding=embeddings
)
#将 vectorstore 转换为一个检索器对象，后续可以通过它快速查询与输入文本相似的条目（基于向量相似度）。
retriever = vectorstore.as_retriever()

In [20]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough
# 并行处理检索和问题
setup_and_retrieval = RunnableParallel(
    {
        "context":retriever,                 #通过 retriever 检索到的相关文本（例如 ["小明在华为工作"]）
        "question":RunnablePassthrough()     #直接传递用户的问题（如 "华为是做什么的？"）。
    }
)
# 构建完整处理链
chain = setup_and_retrieval | prompt | llm

In [23]:
response = chain.invoke("小明在哪里工作？")
output = response.content  # 直接取 content 部分
output

'根据提供的文档内容，小明在华为工作。'

In [ ]:
#如果有多轮对话
outputs = [msg.content for msg in chain.invoke("小明在哪里工作？")]
print(outputs[0])  # 取第一个 content

# 以下将网页文本导到docs_transformed中，以便将来生成本地知识库。

In [ ]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://blog.csdn.net/alva_xu/article/details/147500356", "https://blog.csdn.net/alva_xu/article/details/147456330"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()
docs

In [28]:
from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
docs_transformed = html2text.transform_documents(docs)

docs_transformed

[Document(metadata={'source': 'https://blog.csdn.net/alva_xu/article/details/147500356', 'title': 'LangChain 中主流的 RAG 实现方式_实现rag的方式-CSDN博客', 'description': '文章浏览阅读1.1k次，点赞12次，收藏31次。支持图片、表格等非文本数据，需结合 OCR（如 Tesseract）和专用解析器（如 DeepDoc）。：文档需预先处理好分块  ，该方法的result是字符串，没有metadata信息。：封装文档加载→分块→向量化→存储→检索全流程，适合快速原型开发。：通过设计模板控制生成风格，强制 LLM 基于上下文回答，减少幻觉。：结合语义搜索（向量相似度）与关键词检索（BM25），提升召回率。：通过迭代检索优化结果，例如先检索大纲再定位细节。：提取检索文档中的关键片段，减少无关信息输入。_实现rag的方式', 'language': 'zh-CN'}, page_content='# LangChain 中主流的 RAG 实现方式\n\nalpha xu 于 2025-04-25 10:00:51 发布\n\n阅读量1.1k 收藏 31\n\n点赞数 12\n\n文章标签： langchain 数据库 python 人工智能 自然语言处理 rag\n\n版权声明：本文为博主原创文章，遵循 CC 4.0 BY-SA 版权协议，转载请附上原文出处链接和本声明。\n\n本文链接：https://blog.csdn.net/alva_xu/article/details/147500356\n\n版权\n\n#### 文章目录\n\n  * **一、基础流程实现**\n  *     * 1\\. **全自动索引构建（VectorstoreIndexCreator）**\n    * 2\\. **标准问答链（RetrievalQA）**\n    * 3\\. **Document Chain + 手动检索**\n    * 4\\. **load_qa_chain（传统方式）**\n  * **二、高级定制化实现**\n  *     * 1\\. **自定义提示工程**(RetrievalQA)\n    * 2\\. 

In [29]:
from langchain_community.document_transformers import BeautifulSoupTransformer

bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(
    docs, tags_to_extract=["p", "h3"]
)

docs_transformed

[Document(metadata={'source': 'https://blog.csdn.net/alva_xu/article/details/147500356', 'title': 'LangChain 中主流的 RAG 实现方式_实现rag的方式-CSDN博客', 'description': '文章浏览阅读1.1k次，点赞12次，收藏31次。支持图片、表格等非文本数据，需结合 OCR（如 Tesseract）和专用解析器（如 DeepDoc）。：文档需预先处理好分块  ，该方法的result是字符串，没有metadata信息。：封装文档加载→分块→向量化→存储→检索全流程，适合快速原型开发。：通过设计模板控制生成风格，强制 LLM 基于上下文回答，减少幻觉。：结合语义搜索（向量相似度）与关键词检索（BM25），提升召回率。：通过迭代检索优化结果，例如先检索大纲再定位细节。：提取检索文档中的关键片段，减少无关信息输入。_实现rag的方式', 'language': 'zh-CN'}, page_content='1. 全自动索引构建（VectorstoreIndexCreator） 原理 ：封装文档加载→分块→向量化→存储→检索全流程，适合快速原型开发。通过指定向量 存储类型（如 FAISS）和嵌入模型，自动完成索引构建。  特点 ：自动处理文本分块和向量索引，但灵活性较低  代码示例 ： 2. 标准问答链（RetrievalQA） 原理 ：基于内置模板实现检索增强生成，支持 stuff （文档拼接）、 map_reduce （分块处理）等链类型。  优化点 ：通过 search_kwargs 控制检索数量和质量（如过滤低分文档）。  代码示例 ：  内置默认模板 : LangChain 的 RetrievalQA 默认使用以下隐式模板（未显式声明时自动应用）： 特点 ： 3. Document Chain + 手动检索 原理 ：分离检索与生成步骤，适合需要预处理文档的场景  优势 ：可手动过滤/排序检索结果 4. load_qa_chain（传统方式） 原理 ：直接传入文档列表，适合小规模数据  注意 ：文档需预先处理好分块 ，该方法的result是字符串，没有metadata信息。 1. 自定义提示工程 (RetrievalQA) 原理 ：通过设